In [1]:
import pickle
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
from datetime import datetime

from sklearn.preprocessing import normalize as scikit_normalize

from evaluation import plot_history
from evaluation import rmse_report

from sampling import UnderSampler

from fasttext_embedding import FastTextEmbeddingBag

%matplotlib inline
%load_ext tensorboard

In [2]:
DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='tok'
TB_LOG_DIR='/home/kvassay/data/z/log/E3/scalars/'
FASTTEXT='/home/kvassay/data/z/models/fasttext/sent_tok_lc_100.bin'

In [3]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    train,dev,_=pickle.load(f)

CPU times: user 5.19 s, sys: 1.26 s, total: 6.45 s
Wall time: 6.43 s


In [4]:
fasttext=FastTextEmbeddingBag(FASTTEXT)

In [102]:
def extract_features2(sample, fasttext):
    
    vecs_summary=fasttext.forward([x.lower() for x in sample['summary']])
    vecs_text = fasttext.forward([x.lower() for x in sample['text']])
    avg_summary = np.mean(vecs_summary,axis=0)
    avg_text = np.mean(vecs_text,axis=0)
    return scikit_normalize(np.concatenate((avg_summary,avg_text), axis=0).reshape(1,-1))